# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [5]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                querycounts[s['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')
        
def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        if 'step' in logmessage and isinstance(logmessage['step'],list):
            for s in logmessage['step']:
                if s['url']==url:
                    print(s)

In [6]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [7]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [10]:
pwd

'/Users/priyash/Documents/GitHub/robogallery/user_queries'

In [15]:
standup_json='/Users/priyash/Documents/GitHub/robogallery/standups/StandupDefinitions/standup_3.json'

In [16]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [17]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 3. Named Thing

What things are related to Acrocyanosis (UMLS:C0221347)

Github Issue: https://github.com/NCATSTranslator/testing/issues/12


The query as run through the ARS:

In [18]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "UMLS:C0221347",
                    "category": "biolink:PhenotypicFeature"
                },
                "n1": {
                    "category": "biolink:NamedThing"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [19]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [20]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-16 00:00:00

Done ara-aragorn 340
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=8c6ae8db-1af9-49ae-a9f5-99181bd3dd4b


### 2021-03-17 00:00:00

Error ara-aragorn 0
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=58846701-a542-487c-8602-ef7ff04fb7fa


## Strider Direct

In [21]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 0 results in 0:00:28.245458.


### Provenance

In [22]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count


### Queried sources

In [23]:
print_queried_sources(strider_result)

https://automat.renci.org/covidkopkg/reasonerapi (99 times)
https://automat.renci.org/robokopkg/reasonerapi (81 times)
https://automat.renci.org/ctd/reasonerapi (11 times)
https://automat.renci.org/cord19-scibite/reasonerapi (18 times)
https://automat.renci.org/covid-phenotypes/reasonerapi (3 times)
https://automat.renci.org/cord19-scigraph/reasonerapi (30 times)
https://automat.renci.org/mychem/reasonerapi (6 times)
https://automat.renci.org/ontological-hierarchy/reasonerapi (8 times)
https://automat.renci.org/uberongraph/reasonerapi (99 times)
https://automat.renci.org/pharos/reasonerapi (2 times)
https://automat.renci.org/biolink/reasonerapi (11 times)
https://automat.renci.org/hmdb/reasonerapi (2 times)
https://automat.renci.org/hetio/reasonerapi (5 times)
http://chp.thayer.dartmouth.edu/query/ (2 times)
https://trapi.monarchinitiative.org/query (47 times)
https://translator.broadinstitute.org/molepro/trapi/v1.0/query (1 times)
https://stars-app.renci.org/sparql-kp/query (57 times)

### Errors

In [24]:
print_errors(strider_result)

In [27]:
strider_result

{'message': {'query_graph': {'nodes': {'n1': {'id': None,
     'category': ['biolink:NamedThing'],
     'is_set': False},
    'n0': {'id': ['UMLS:C0221347'],
     'category': ['biolink:PhenotypicFeature'],
     'is_set': False}},
   'edges': {'e01': {'subject': 'n0',
     'object': 'n1',
     'predicate': None,
     'relation': None}}},
  'knowledge_graph': {'nodes': {}, 'edges': {}},
  'results': []},
 'logs': [{'timestamp': '2021-05-20T17:41:58.705298',
   'level': 'WARNING',
   'code': None,
   'message': "Could not find preferred prefixes for at least one of: ['biolink:Disease', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature', 'biolink:BiologicalEntity', 'biolink:NamedThing', 'biolink:Entity']"},
  {'timestamp': '2021-05-20T17:41:58.687693',
   'level': 'WARNING',
   'code': None,
   'message': "Could not find preferred prefixes for at least one of: ['biolink:Disease', 'biolink:Disease', 'biolink:DiseaseOrPhenotypicFeature', 'biolink:BiologicalEntity', 'biolink:NamedThing',

### Results

In [28]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

Enter Assessment Here

## ARAGORN 

In [26]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

Error, no result field


In [20]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

In [27]:
tq={
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "id": "HP:0033127",
                    "category": "biolink:PhenotypicFeature"
                },
                "n2": {
                    "category": "biolink:Disease"
                }
            },
            "edges": {
                "e02": {
                    "subject": "n1",
                    "object": "n2",
                    "predicate": "biolink:affected_by"
                }
            }
        }
    }
}

In [29]:
r = automat('uberongraph',tq)

200


### ARAGORN Assessment

How did we do?